In [1]:
!python -m venv myenv
!source myenv/bin/activate

In [2]:
!pip install datasets transformers==4.18.0 sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 780.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 65.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 38.1 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kaggle-environments 1.14.11 requires transformers>=4.33.1, but you have transformers 4.18.0 which is incompatible.


In [3]:
# from datasets import *
# from transformers import *
# from tokenizers import *

import datasets
import transformers
import tokenizers
import os
import json

In [4]:
# download and prepare cc_news dataset
dataset = datasets.load_dataset("sarwarshafee8709809365/Synthetic_Story_Generated_By_Gemini", split="train")

Generating train split:   0%|          | 0/184819 [00:00<?, ? examples/s]

In [5]:
d = dataset.train_test_split(test_size=0.01)
d["train"], d["test"]

(Dataset({
     features: ['text'],
     num_rows: 182970
 }),
 Dataset({
     features: ['text'],
     num_rows: 1849
 }))

In [6]:
for t in d["train"]["text"][:3]:
  print(t)
  print("="*50)

একটি ঘন জঙ্গলের প্রান্তে একটি ছোট গ্রাম ছিল। সেখানে জগন্নাথ নামে এক দরিদ্র কৃষক বাস করত। তাঁর স্ত্রী ও দুটি ছোট সন্তান ছিল। জগন্নাথ খুব পরিশ্রমী ছিল। সে খুব ভোরে ঘুম থেকে উঠে কাজে বেরিয়ে পড়ত। কিন্তু তার সব পরিশ্রম সত্ত্বেও তাদের খেতে পরতে খুব অসুবিধা হত।

একদিন জগন্নাথ জঙ্গলে কাঠ কাটতে গেছিল। কাঠ কাটতে কাটতে সে খুব ক্লান্ত হয়ে পড়ল। তাই সে একটি গাছের তলায় বসে কিছুক্ষণের জন্য বিশ্রাম নিল। হঠাৎ তার চোখে পড়ল একটি বড় পাথর। পাথরের উপর কিছু অদ্ভুত অক্ষর খোদাই করা ছিল। জগন্নাথ খুব কৌতূহলী হয়ে পাথরটা ঘষে ঘষে পরিষ্কার করল।

পাথরটা পরিষ্কার হওয়ার পর জগন্নাথ দেখল, সেখানে লেখা আছে, "এই পাথরের নীচে লুকিয়ে আছে একটি ধন।" জগন্নাথ খুব খুশি হল। সে ইদিক ওদিক তাকিয়ে দেখল, কেউ নেই। তারপর সে পাথরটা তুলে ফেলল। পাথরের নীচে সত্যিই একটি বড় বাক্স ছিল।

জগন্নাথ সেই বাক্সটা তুলে বাড়িতে নিয়ে এল। বাক্সটা খুলে দেখল, তার মধ্যে রয়েছে স্বর্ণের মুদ্রা, রতন এবং অনেক মূল্যবান জিনিস। জগন্নাথ ও তার পরিবারের খুশির সীমা রইল না। তারা এবার থেকে আর দরিদ্র নেই। তারা সুখে-শান্তিতে জীবন কাটাল।
এক পুকুরের পাশে আরাম করে 

In [7]:
# if you want to train the tokenizer from scratch (especially if you have custom
# dataset loaded as datasets object), then run this cell to save it as files
# but if you already have your custom data as text files, there is no point using this
def dataset_to_text(dataset, output_filename="data.txt"):
  """Utility function to save dataset text to disk,
  useful for using the texts to train the tokenizer
  (as the tokenizer accepts files)"""
  with open(output_filename, "w") as f:
    for t in dataset["text"]:
      print(t, file=f)

# save the training set to train.txt
dataset_to_text(d["train"], "train.txt")
# save the testing set to test.txt
dataset_to_text(d["test"], "test.txt")

In [8]:
special_tokens = [
  "[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]", "<S>", "<T>"
]
# if you want to train the tokenizer on both sets
# files = ["train.txt", "test.txt"]
# training the tokenizer on the training set
files = ["train.txt"]
# 30,522 vocab is BERT's default vocab size, feel free to tweak
vocab_size = 20_000
# maximum sequence length, lowering will result to faster training (when increasing batch size)
max_length = 512
# whether to truncate
truncate_longer_samples = True

In [9]:
# initialize the WordPiece tokenizer
tokenizer = tokenizers.BertWordPieceTokenizer()
# train the tokenizer
tokenizer.train(files=files, vocab_size=vocab_size, special_tokens=special_tokens)
# enable truncation up to the maximum 512 tokens
tokenizer.enable_truncation(max_length=max_length)

In [10]:
model_path = "shafee-bert"
# make the directory if not already there
if not os.path.isdir(model_path):
  os.mkdir(model_path)

In [11]:
# save the tokenizer
tokenizer.save_model(model_path)

['shafee-bert/vocab.txt']

In [12]:
# dumping some of the tokenizer config to config file,
# including special tokens, whether to lower case and the maximum sequence length
with open(os.path.join(model_path, "config.json"), "w") as f:
  tokenizer_cfg = {
      "do_lower_case": True,
      "unk_token": "[UNK]",
      "sep_token": "[SEP]",
      "pad_token": "[PAD]",
      "cls_token": "[CLS]",
      "mask_token": "[MASK]",
      "model_max_length": max_length,
      "max_len": max_length,
  }
  json.dump(tokenizer_cfg, f)

In [13]:
# when the tokenizer is trained and configured, load it as BertTokenizerFast
tokenizer = transformers.BertTokenizerFast.from_pretrained(model_path)

In [14]:
def encode_with_truncation(examples):
  """Mapping function to tokenize the sentences passed with truncation"""
  return tokenizer(examples["text"], truncation=True, padding="max_length",
                   max_length=max_length, return_special_tokens_mask=True)

def encode_without_truncation(examples):
  """Mapping function to tokenize the sentences passed without truncation"""
  return tokenizer(examples["text"], return_special_tokens_mask=True)

# the encode function will depend on the truncate_longer_samples variable
encode = encode_with_truncation if truncate_longer_samples else encode_without_truncation

# tokenizing the train dataset
train_dataset = d["train"].map(encode, batched=True)
# tokenizing the testing dataset
test_dataset = d["test"].map(encode, batched=True)

if truncate_longer_samples:
  # remove other columns and set input_ids and attention_mask as PyTorch tensors
  train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
  test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])
else:
  # remove other columns, and remain them as Python lists
  test_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])
  train_dataset.set_format(columns=["input_ids", "attention_mask", "special_tokens_mask"])

Map:   0%|          | 0/182970 [00:00<?, ? examples/s]

Map:   0%|          | 0/1849 [00:00<?, ? examples/s]

In [15]:
from itertools import chain
# Main data processing function that will concatenate all texts from our dataset and generate chunks of
# max_seq_length.
# grabbed from: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
def group_texts(examples):
    # Concatenate all texts.
#     print("examples structure:", {k: type(v) for k, v in examples.items()})
#     examples.pop('row_number', None)
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= max_length:
        total_length = (total_length // max_length) * max_length
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + max_length] for i in range(0, total_length, max_length)]
        for k, t in concatenated_examples.items()
    }
    return result


# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a
# remainder for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value
# might be slower to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map
if not truncate_longer_samples:
  train_dataset = train_dataset.map(group_texts, batched=True,
                                    desc=f"Grouping texts in chunks of {max_length}")
  test_dataset = test_dataset.map(group_texts, batched=True,
                                  desc=f"Grouping texts in chunks of {max_length}")
  # convert them from lists to torch tensors
  train_dataset.set_format("torch")
  test_dataset.set_format("torch")

In [16]:
len(train_dataset), len(test_dataset)

(182970, 1849)

In [17]:
# initialize the model with the config
model_config = transformers.BertConfig(vocab_size=vocab_size, max_position_embeddings=max_length)
model = transformers.BertForMaskedLM(config=model_config)

In [18]:
# initialize the data collator, randomly masking 20% (default is 15%) of the tokens for the Masked Language
# Modeling (MLM) task
# data_collator = transformers.DataCollatorForLanguageModeling(
#     tokenizer=tokenizer, mlm=True, mlm_probability=0.2
# )

data_collator = transformers.DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.2,
    # Make sure all tensors are long type
    return_tensors='pt'
)

2024-06-15 19:53:08.387895: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-15 19:53:08.388007: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-15 19:53:08.544969: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
training_args = transformers.TrainingArguments(
    output_dir=model_path,          # output directory to where save model checkpoint
    evaluation_strategy="steps",    # evaluate each `logging_steps` steps
    overwrite_output_dir=True,
    num_train_epochs=10,            # number of training epochs, feel free to tweak
    per_device_train_batch_size=10, # the training batch size, put it as high as your GPU memory fits
    gradient_accumulation_steps=8,  # accumulating the gradients before updating the weights
    per_device_eval_batch_size=64,  # evaluation batch size
    logging_steps=1000,             # evaluate, log and save model checkpoints every 1000 step
    save_steps=1000,
    report_to="wandb"
    # load_best_model_at_end=True,  # whether to load the best model (in terms of loss) at the end of training
    # save_total_limit=3,           # whether you don't have much space so you let only 3 model weights saved in the disk
)

In [ ]:
# initialize the trainer and pass everything to it
trainer = transformers.Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)

In [ ]:
# train the model
trainer.train()